In [7]:
from bs4 import BeautifulSoup
import requests

head_Html_lotto='https://www.taiwanlottery.com.tw/lotto/dailycash/history.aspx'
res = requests.get(head_Html_lotto, timeout=30)
print(res.text)



<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8"/>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>今彩539 各期獎號與開獎結果</title>    
     <link href="/css1.css" rel="stylesheet" type="text/css" /> 
<script type="text/JavaScript">

<!--

if (navigator.appName.indexOf("Internet Explorer") != -1) 

document.onmousedown = noSourceExplorer;

function noSourceExplorer()

{

if (event.button == 2 | event.button == 3)

{

alert("版權所有 台灣彩券");

//location.replace("http://www.taiwanlottery.com.tw/");

}

}

-->

</script>

<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insert

In [8]:
winning_Numbers_Sort_lotto = ['D539Control_history1_dlQuery_No1_0',
'D539Control_history1_dlQuery_No2_0',
'D539Control_history1_dlQuery_No3_0',
'D539Control_history1_dlQuery_No4_0',
'D539Control_history1_dlQuery_No5_0',
'D539Control_history1_dlQuery_D539_DDate_0'
]

def search_winning_numbers(css_class):
    global winning_Numbers_Sort_lotto
    if(css_class != None):
        for i in range(len(winning_Numbers_Sort_lotto )):
            if winning_Numbers_Sort_lotto [i] in css_class:
                return css_class              
head_Html_lotto='https://www.taiwanlottery.com.tw/lotto/dailycash/history.aspx'
res = requests.get(head_Html_lotto, timeout=30)
#print(res.text)
soup = BeautifulSoup(res.text,'lxml')
#print(soup2.prettify())
header_Info = soup.find_all(id=search_winning_numbers)
for item in header_Info:
    print(item)

<span id="D539Control_history1_dlQuery_D539_DDate_0">112/02/18</span>
<span id="D539Control_history1_dlQuery_No1_0">08</span>
<span id="D539Control_history1_dlQuery_No2_0">12</span>
<span id="D539Control_history1_dlQuery_No3_0">18</span>
<span id="D539Control_history1_dlQuery_No4_0">20</span>
<span id="D539Control_history1_dlQuery_No5_0">32</span>


In [9]:
def parse_tw_lotto_html(data_Info,number_count):  
    data_Info_List = []
    data_Info_Dict = {}
    tmp_index = 0
    for index  in range(len(data_Info)) :
        if (index == 0):
			#用data_Info[index].text 即可取得獎號
            data_Info_List.append(data_Info[index].text)  
        else:
            if(index % number_count != 0):
                data_Info_List.append(data_Info[index].text)
            else:
                data_Info_Dict[str(tmp_index)] = list(data_Info_List)
                data_Info_List= []
                data_Info_List.append(data_Info[index].text)
                tmp_index = tmp_index+1
        data_Info_Dict[str(tmp_index)] = list(data_Info_List)
    return data_Info_List,data_Info_Dict

In [10]:
head_Html_lotto='https://www.taiwanlottery.com.tw/lotto/dailycash/history.aspx'
res = requests.get(head_Html_lotto, timeout=30)
soup = BeautifulSoup(res.text,'lxml')
header_Info = soup.find_all(id=search_winning_numbers)
data_Info_List,data_Info_Dict  = parse_tw_lotto_html(header_Info,7)    
print(data_Info_Dict)

{'0': ['112/02/18', '08', '12', '18', '20', '32']}


In [11]:
import sqlite3
import time

yyyymmdd = str(int(data_Info_Dict['0'][0][:3])+1911) + data_Info_Dict['0'][0][3:9]


conn = sqlite3.connect('lottery_project_DB.db')
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS lottery_numbers (day DATE PRIMARY KEY , number1 TEXT, number2 TEXT, number3 TEXT, number4 TEXT, number5 TEXT, yyyymmdd TEXT)')
cur.execute("INSERT INTO lottery_numbers (day, number1, number2, number3, number4, number5,\
            yyyymmdd) \
            values('{}',{},{},{},{},{},'{}');".format(data_Info_Dict['0'][0], \
        data_Info_Dict['0'][1],data_Info_Dict['0'][2],data_Info_Dict['0'][3],data_Info_Dict['0'][4],data_Info_Dict['0'][5],\
            yyyymmdd))


conn.commit()
conn.close()

IntegrityError: UNIQUE constraint failed: lottery_numbers.day

In [12]:
conn.close()